# Manejo de Datasets con Pandas

Este notebook tiene como objetivo introducir el manejo de **datasets con la librería Pandas**,
simulando operaciones similares a las de una base de datos SQL: `SELECT`, `WHERE`, `JOIN`, `GROUP BY`.

**Objetivo:** aprender a cargar, filtrar, unir y agrupar datos usando Pandas,
aplicando estos conocimientos a modelos de datos como asistencias, stock o consultas médicas.


## Importación de Pandas y carga de datos

Primero importamos la librería Pandas y creamos algunos datasets de ejemplo.
<BR>
Vamos a seguir en línea con el schema de Postgres, una tabla para estudiantes y otra para asistencias.


En caso de tener que importar un archivo 'csv' o 'json' pueden hacerlo con:

* df = pd.read_csv("ejemplo.csv")
* df = pd.read_json("archivo.json")

In [ ]:
# Importar Pandas
import pandas as pd

In [ ]:
# Creamos el dataset estudiante con algunos datos:
estudiantes = pd.DataFrame({
    'id_estudiante': [1, 2, 3,],
    'nombre': ['Ana', 'Luis', "Sofía"],
    'apellido': ['Kunst', 'Martins', 'Ripol'],
    'legajo': ['1568', '2649', '1523']
})

estudiantes

In [ ]:
# Creamos el dataset asistencias con algunos datos:
asistencias = pd.DataFrame({
    'id_asistencia': [101, 102, 103, 104, 105],
    'id_estudiante': [1, 2, 1, 3, 2],
    'asignatura': ['Algoritmos', 'Marketing', 'Algoritmos', 'Progra I', 'Marketing'],
    'fecha': ['2025-03-01', '2025-04-01', '2025-04-09', '2025-05-08', '2025-05-15'],
    'estado': ["P", "A", "M", "P", "P"]
})

asistencias

## 🔍 Exploración básica de DataFrames

Podemos inspeccionar la estructura y contenido de los DataFrames.


In [ ]:
# Primeras filas
estudiantes.head()

In [ ]:
# Información general
estudiantes.info()

In [ ]:
# Listar las columnas
estudiantes.columns

## Análisis y conversión de tipo de datos

### dtypes

In [ ]:
# A diferencia de Python nativo, en Pandas podemos forzar un tipo de dato.
# La propiedad es dtype
asistencias.dtypes

### astype()

In [ ]:
# Convertir str a int o int a str con astype
asistencias['id_estudiante'] = asistencias['id_estudiante'].astype(str)
asistencias.dtypes
#astype() admite, int, float, str

### to_datetime()

In [ ]:
# Vamos a convertir el dtype de fecha.
asistencias['fecha'].dtype

asistencias['fecha'].sample() # YYY-MM-DD

asistencias['fecha'] = pd.to_datetime(asistencias['fecha'])
# se admite format="%d/%m/%Y", erros="coerce"

asistencias['fecha'].dtype

## Selección y filtrado de columnas y filas

Simulamos un `SELECT` y un `WHERE` en SQL.


In [ ]:
# Seleccionar columnas específicas
estudiantes[['id_estudiante', 'nombre']]


In [ ]:
# Identifcar valores únicos
asistencias['asignatura'].unique()

In [ ]:
# Contar valores unicos
asistencias['asignatura'].value_counts()

In [ ]:
# Filtrar filas (WHERE asignatura = 'Matemática Discreta')
asistencias[asistencias['asignatura'] == 'Marketing']


In [ ]:
# Filtrar filas (WHERE asignatura similar 'Matemática Discreta')
asistencias[asistencias['asignatura'].str.lower().str.contains("mark", case=False)]

In [ ]:
# Filtrar con query
asistencias.query("asignatura == 'Algoritmos'")



In [ ]:
asistencias.loc[asistencias["estado"] == "P", ["asignatura"]]

In [ ]:
# Acceder al mes o año para filtros mas amplios
asistencias[asistencias['fecha'].dt.month == 4]

In [ ]:
# Acceder al mes o año para filtros mas amplios con &
asistencias[(asistencias['fecha'].dt.month == 4) & (asistencias['asignatura']=='Algoritmos')]

## 📊 Ordenamiento de datos

Simula un `ORDER BY`.


In [ ]:
# Ordenar estudiantes por nombre
estudiantes.sort_values('nombre')


In [ ]:
# Ordenar asistencias por asignatura descendiente
asistencias.sort_values(by="asignatura", ascending=False)

In [ ]:
# Ordenar por asignatura y por estado
asistencias.sort_values(['fecha', 'estado'])


## 🔗 Combinación de DataFrames (JOIN)

Simula un `JOIN` entre tablas SQL.


In [ ]:
# INNER JOIN: estudiantes con sus asistencias
join_ea = pd.merge(estudiantes, asistencias, on='id_estudiante', how='inner')
join_ea


## 📈 Agrupamiento y agregación (GROUP BY)

Simula un `GROUP BY` de SQL.


In [ ]:
# Cantidad de asistencias por carrera
# Primero filtramos los presentes
reporte = join_ea[asistencias['estado'] == 'P']
reporte = reporte.groupby('asignatura')['estado'].count().reset_index()
reporte

In [ ]:
# Cantidad de asistencias por estudiante
# Primero filtramos los presentes
reporte = join_ea[asistencias['estado'] == 'P']
reporte = reporte.groupby(['nombre', 'apellido'], as_index=False)['estado'].count().reset_index()
reporte

In [ ]:
# Cantidad de asistencias por estudiante (usando otros métodos de agregación)
# Primero filtramos los presentes
reporte = join_ea[asistencias['estado'] == 'P']
# reporte = reporte.groupby(['nombre', 'apellido'], as_index=False).agg(
#     asistencia = ('estado', 'count')
# )
reporte = reporte.groupby(['nombre', 'apellido'], as_index=False).agg(
    {'estado': 'count'}
)
reporte

## 🧩 Ejercicio final

1. Crea DataFrames para otro modelo (por ejemplo, *productos*, *proveedores* y *stock*).
2. Usa `merge()` para generar un reporte que relacione productos y proveedores.
3. Usa `groupby()` para obtener el total de stock por proveedor.
4. Guarda el resultado en un archivo CSV con `to_csv('reporte.csv', index=False)`.
